In [1]:
import imaplib
import email
import os
import pytesseract
import cv2
import numpy as np
from PIL import Image

# Update this with your email credentials
EMAIL = "tanuja.ch076@gmail.com"
PASSWORD = "omip uivi tqxc owpa"

# 📌 Connect to Gmail IMAP Server
def connect_to_gmail():
    mail = imaplib.IMAP4_SSL("imap.gmail.com")
    mail.login(EMAIL, PASSWORD)
    mail.select("inbox")
    return mail

# 📩 Fetch the latest email from Screentime/Digital Wellbeing
def fetch_screentime_email():
    mail = connect_to_gmail()
    
    # Search for emails from "Screentime" or "Digital Wellbeing"
    result1, data1 = mail.search(None, 'FROM "Screentime"')
    result2, data2 = mail.search(None, 'FROM "Digital Wellbeing"')
    
    email_ids = data1[0].split() + data2[0].split()

    if not email_ids:
        print("No screentime emails found!")
        return None

    latest_email_id = email_ids[-1]  # Get the latest email
    result, email_data = mail.fetch(latest_email_id, "(RFC822)")

    for response_part in email_data:
        if isinstance(response_part, tuple):
            msg = email.message_from_bytes(response_part[1])
            for part in msg.walk():
                if part.get_content_maintype() == "multipart":
                    continue
                if part.get_content_subtype() in ["jpeg", "png", "jpg"]:
                    filename = part.get_filename()
                    if filename:
                        filepath = os.path.join(os.getcwd(), filename)
                        with open(filepath, "wb") as f:
                            f.write(part.get_payload(decode=True))
                        print(f"✅ Image saved: {filepath}")
                        return filepath

    print("⚠️ No images found in email.")
    return None

# 🖼️ Extract text from the screen time graph
def extract_text_from_graph(image_path):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Preprocess image for better OCR results
    gray = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    
    text = pytesseract.image_to_string(gray)
    return text

# 📊 Analyze extracted text for specific apps
def analyze_screen_time(extracted_text):
    apps_to_track = ["Netflix", "Spotify", "Swiggy"]
    app_usage = {}

    for app in apps_to_track:
        if app.lower() in extracted_text.lower():
            # Extract the usage time (basic regex-like search)
            lines = extracted_text.split("\n")
            for line in lines:
                if app.lower() in line.lower():
                    app_usage[app] = line
                    break

    return app_usage

# 📝 Store extracted screen time
def store_screen_time(app_usage):
    with open("screen_time_report.txt", "w") as f:
        for app, data in app_usage.items():
            f.write(f"{app}: {data}\n")
            print(f"Stored {app} screen time!")

# 🔥 Main Execution
image_path = fetch_screentime_email()
if image_path:
    extracted_text = extract_text_from_graph(image_path)
    app_usage = analyze_screen_time(extracted_text)
    
    if app_usage:
        store_screen_time(app_usage)
    else:
        print("⚠️ No relevant apps found in the extracted data.")


✅ Image saved: C:\Users\Tanuja\Desktop\AIPro\screentimeusage.png
Stored Netflix screen time!
Stored Spotify screen time!


In [2]:
pip install pytesseract


  Obtaining dependency information for pytesseract from https://files.pythonhosted.org/packages/7a/33/8312d7ce74670c9d39a532b2c246a853861120486be9443eebf048043637/pytesseract-0.3.13-py3-none-any.whl.metadata
Note: you may need to restart the kernel to use updated packages.


In [11]:
import imaplib
import email
import os
import cv2
import pytesseract
import re
import matplotlib.pyplot as plt

# ✅ Configure Gmail credentials
EMAIL = "tanuja.ch076@gmail.com"
PASSWORD = "omip uivi tqxc owpa"

# ✅ Connect to Gmail
def connect_to_gmail():
    try:
        mail = imaplib.IMAP4_SSL("imap.gmail.com")
        mail.login(EMAIL, PASSWORD)
        mail.select("inbox")
        print("✅ Connected to Gmail.")
        return mail
    except Exception as e:
        print(f"❌ Gmail connection failed: {e}")
        return None

# ✅ Fetch latest email with screen time image
def fetch_screentime_email():
    mail = connect_to_gmail()
    if not mail:
        return None

    result1, data1 = mail.search(None, 'FROM "Screentime"')
    result2, data2 = mail.search(None, 'FROM "Digital Wellbeing"')

    email_ids = data1[0].split() + data2[0].split()
    if not email_ids:
        print("❌ No screentime emails found!")
        return None

    latest_email_id = email_ids[-1]
    result, email_data = mail.fetch(latest_email_id, "(RFC822)")

    for response_part in email_data:
        if isinstance(response_part, tuple):
            msg = email.message_from_bytes(response_part[1])
            for part in msg.walk():
                filename = part.get_filename()
                if filename and filename.endswith((".jpg", ".jpeg", ".png")):
                    filepath = os.path.join(os.getcwd(), filename)
                    with open(filepath, "wb") as f:
                        f.write(part.get_payload(decode=True))
                    print(f"✅ Image saved: {filepath}")
                    return filepath

    print("⚠️ No images found in email.")
    return None

# ✅ Extract text from screen time image
def extract_text_from_graph(image_path):
    img = cv2.imread(image_path)
    if img is None:
        print("❌ Failed to load image.")
        return ""

    # 🔍 Improve image for better OCR
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)  # Reduce noise
    gray = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

    # ✅ Save processed image for debugging
    cv2.imwrite("processed_image.png", gray)

    # 🔍 Extract text
    text = pytesseract.image_to_string(gray)

    print("🔍 Extracted OCR Text:\n", text)  # Debugging print
    return text.lower()

# ✅ Extract screen time for specific apps
def analyze_screen_time(extracted_text):
    apps_to_track = ["Netflix", "Spotify", "Swiggy"]
    app_usage = {}

    for app in apps_to_track:
        match = re.search(rf"{app}\s*(\d+)\s*(h|hours|hrs|H)?\s*(\d+)?\s*(m|minutes|mins|M)?", 
                          extracted_text, re.IGNORECASE)
        print(f"🔎 Checking {app}: Match = {match}")  # Debugging print

        if match:
            hours = int(match.group(1)) if match.group(1) else 0
            minutes = int(match.group(3)) if match.group(3) else 0
            total_time_hours = hours + (minutes / 60)  # Convert to hours
            app_usage[app.capitalize()] = round(total_time_hours, 2)

    return app_usage

# ✅ Store extracted screen time in a file
def store_screen_time(app_usage):
    with open("screen_time_report.txt", "w") as f:
        for app, hours in app_usage.items():
            f.write(f"{app}: {hours} hours\n")
            print(f"Stored {app}: {hours} hours")

# ✅ Plot screen time usage
def plot_screentime_usage(data):
    if not data:
        print("❌ No data to plot!")
        return

    apps = list(data.keys())
    usage = list(data.values())

    plt.figure(figsize=(8, 5))
    bars = plt.bar(apps, usage, color=["red", "blue", "green"])

    for bar in bars:
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height(), f"{bar.get_height()}h",
                 ha='center', color='white', fontsize=12, fontweight='bold')

    plt.xlabel("Apps")
    plt.ylabel("Screen Time (Hours)")
    plt.title("Screen Time Usage (Netflix, Spotify, Swiggy)")
    plt.show()

# 🔥 **Main Execution**
def main():
    image_path = fetch_screentime_email()

    if not image_path:
        print("❌ No image found. Exiting...")
        return

    extracted_text = extract_text_from_graph(image_path)
    if not extracted_text.strip():
        print("❌ No text extracted. OCR failed!")
        return

    app_usage = analyze_screen_time(extracted_text)
    if not app_usage:
        print("❌ No relevant app usage data found!")
        return

    print("📊 Extracted Screen Time Data:", app_usage)
    store_screen_time(app_usage)
    plot_screentime_usage(app_usage)

# ✅ Run the script
if __name__ == "__main__":
    main()


✅ Connected to Gmail.
✅ Image saved: C:\Users\Tanuja\Desktop\AIPro\screentimeusage.png
🔍 Extracted OCR Text:
 Screen Time (Hours per Month)

140

Digital WellbeIng: Monthly Screen Time Usage

Spotify Swiggy Gals WhatsApp

Instagram

🔎 Checking Netflix: Match = None
🔎 Checking Spotify: Match = None
🔎 Checking Swiggy: Match = None
❌ No relevant app usage data found!


In [12]:
text = extract_text_from_graph("C:\\Users\\Tanuja\\Desktop\\AIPro\\screentimeusage.png")
print("🔍 Full Extracted Text:\n", text)


🔍 Extracted OCR Text:
 Screen Time (Hours per Month)

140

Digital WellbeIng: Monthly Screen Time Usage

Spotify Swiggy Gals WhatsApp

Instagram

🔍 Full Extracted Text:
 screen time (hours per month)

140

digital wellbeing: monthly screen time usage

spotify swiggy gals whatsapp

instagram

